In [21]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

We use MNIST dataset (handwritten digits 0–9). It has 70,000 images, each 28×28 pixels → flattened into 784 input features.

Dataset size is about 11 MB — small enough for “from scratch” training.

Features (X) are normalized to [0, 1] by dividing by 255, which helps avoid exploding gradients during training.

Labels (y) are converted into one-hot encoded vectors.
Example: digit 3 → [0,0,0,1,0,0,0,0,0,0].
         digit 5 → [0,0,0,0,0,1,0,0,0,0].

In [22]:
print("Loading dataset...")
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

Loading dataset...


In [23]:
X = X / 255.0 

In [24]:
enc = OneHotEncoder(sparse_output=False, categories="auto")
y = enc.fit_transform(y.reshape(-1, 1))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Input layer: 784 nodes (one for each pixel).

Hidden layer: 128 neurons. This number is a hyperparameter (too few = underfitting, too many = slow training).

Output layer: 10 neurons (since MNIST has 10 classes (numbers 0 to 9)).

Learning Rate: 0.02 (Controls how big the update step is. If learning rate is too small the update may take too long to reach optimized value of weights or biases. And if learning rate is too large the update may miss the value and never reach the required optimized value.)

Epochs: 30 (Each epoch is complete training of dataset 1 time by the learning algorithm. So, the dataset is trained 20 
            times with updated values (weight and bias) learned from each epoch to make the model better.)

#Parameters initialized:

    - Weights (W1, W2): Randomly initialized with Xavier initialization to keep signal variance stable across layers.

    - Biases (b1, b2): Initialized to zero.

In [26]:
input_size = 784   # MNIST images 28x28
hidden_size = 128
output_size = 10
lr = 0.03
epochs = 40

In [27]:
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * np.sqrt(1. / input_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * np.sqrt(1. / hidden_size)
b2 = np.zeros((1, output_size))

ReLU (Rectified Linear Unit) for hidden layer:

    - Formula: f(x) = max(0, x).

    - Purpose: introduces non-linearity so the network can learn complex mappings.

    - Derivative: 1 if x>0, otherwise 0. Used during backprop.

Softmax for output layer:

    - Converts raw scores into probabilities: softmax(z_i) = exp(z_i) / Σ exp(z_j)

    - Ensures outputs sum to 1, making them interpretable as probabilities.

Cross-Entropy Loss:

    - Measures the difference between predicted probability distribution and actual labels.

    - Formula: L = -(1/m) Σ y_true * log(y_pred)

    - Penalizes the model heavily when it assigns low probability to the correct class.


Lower loss = better performance.

In [28]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / m

# Forward Propagation:

    Compute weighted sum at hidden layer: Z1 = X·W1 + b1

    Apply activation (ReLU): A1 = ReLU(Z1)

    Compute weighted sum at output layer: Z2 = A1·W2 + b2

    Apply softmax: A2 = softmax(Z2)
    → gives probability distribution over 10 classes.

    This forward pass predicts outputs based on current weights.

# Backward Propagation:

    Output layer error: dZ2 = A2 - y_true
    (difference between predicted probabilities and true labels).

    Gradients for output weights/bias:  dW2 = (A1ᵀ·dZ2)/m
                                        db2 = sum(dZ2)/m

    Hidden layer error: dA1 = dZ2·W2ᵀ
                        dZ1 = dA1 * ReLU'(Z1)
    (applying derivative of ReLU).

    Gradients for hidden weights/bias:  dW1 = (Xᵀ·dZ1)/m
                                        db1 = sum(dZ1)/m

    These gradients tell us how much each weight/bias contributed to the error.

# Parameter Update:

    Gradient Descent is used:   W = W - lr * dW
                                b = b - lr * db

    lr = learning rate (0.01). Controls how big the update step is.

    Repeat across epochs (20 in this case).

In [29]:
for epoch in range(epochs):
    # Forward pass
    Z1 = np.dot(X_train, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)

    # Loss
    loss = cross_entropy(y_train, A2)

    # Backpropagation
    m = X_train.shape[0]
    dZ2 = A2 - y_train
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X_train.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Update parameters
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2

    if epoch % 2 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

Epoch 1/40, Loss: 2.3386
Epoch 3/40, Loss: 2.3041
Epoch 5/40, Loss: 2.2720
Epoch 7/40, Loss: 2.2417
Epoch 9/40, Loss: 2.2128
Epoch 11/40, Loss: 2.1847
Epoch 13/40, Loss: 2.1573
Epoch 15/40, Loss: 2.1302
Epoch 17/40, Loss: 2.1032
Epoch 19/40, Loss: 2.0764
Epoch 21/40, Loss: 2.0496
Epoch 23/40, Loss: 2.0227
Epoch 25/40, Loss: 1.9957
Epoch 27/40, Loss: 1.9686
Epoch 29/40, Loss: 1.9415
Epoch 31/40, Loss: 1.9142
Epoch 33/40, Loss: 1.8869
Epoch 35/40, Loss: 1.8596
Epoch 37/40, Loss: 1.8322
Epoch 39/40, Loss: 1.8049


In [30]:
Z1 = np.dot(X_test, W1) + b1
A1 = relu(Z1)
Z2 = np.dot(A1, W2) + b2
A2 = softmax(Z2)

preds = np.argmax(A2, axis=1)
labels = np.argmax(y_test, axis=1)
acc = np.mean(preds == labels)

print(f"Test Accuracy: {acc*100:.2f}%")

Test Accuracy: 66.54%


At 20 epoch and 0.01 learning rate the accuracy was obtained 23.93%. 

Also the accuracy at 30 epoch and 0.02 learning rate obtained was 48.11%